In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asecnasj2223/20221210.csv
/kaggle/input/asecnasj2223/20221219.csv
/kaggle/input/asecnasj2223/20230122.csv
/kaggle/input/asecnasj2223/20230223.csv
/kaggle/input/asecnasj2223/20230429.csv
/kaggle/input/asecnasj2223/20230419.csv
/kaggle/input/asecnasj2223/20221212.csv
/kaggle/input/asecnasj2223/20230211.csv
/kaggle/input/asecnasj2223/20221022.csv
/kaggle/input/asecnasj2223/20230212.csv
/kaggle/input/asecnasj2223/20221016.csv
/kaggle/input/asecnasj2223/20230409.csv
/kaggle/input/asecnasj2223/20221203.csv
/kaggle/input/asecnasj2223/20230406.csv
/kaggle/input/asecnasj2223/20230307.csv
/kaggle/input/asecnasj2223/20230116.csv
/kaggle/input/asecnasj2223/20230417.csv
/kaggle/input/asecnasj2223/20230110.csv
/kaggle/input/asecnasj2223/20230306.csv
/kaggle/input/asecnasj2223/20221021.csv
/kaggle/input/asecnasj2223/20221012.csv
/kaggle/input/asecnasj2223/20221119.csv
/kaggle/input/asecnasj2223/20230117.csv
/kaggle/input/asecnasj2223/20230115.csv
/kaggle/input/asecnasj2223/20230104.csv


# **Summary of the project**

The project aims to analyze data to determine the locations in Madagascar airspace that experience the highest congestion. It will also identify the times within a 24-hour period that are more prone to congestion and investigate potential causes by examining the correlation between turbulence and flight rules.

# **II-Data:** 
The data we have collected are data from September 2022 to June 2023 which  contains the following column: 
1. Line Number: Identification number for the flight record.
2. Immatriculation: Aircraft registration number.
3. Avion Type: Type of aircraft used for the flight.
4. Turbulence Type (1, 2, 3, 4): Categorization of turbulence intensity.
5. Departure Emplacement (ICAO code): ICAO code representing the departure location.
6. Destination Emplacement (ICAO code): ICAO code representing the destination location.
7. Flight Rule: Flight rules followed, categorized as VFR-VMC (Visual Flight Rules - Visual Meteorological Conditions) or IFR-IMC (Instrument Flight Rules - Instrument Meteorological Conditions).
8. Radioborn Maker - Avion Capacity (COMNAV): Manufacturer of the radio equipment installed in the aircraft for communication and navigation purposes.
9. Radioborn Maker - Avion Capacity (Surveillance): Manufacturer of the radio equipment installed in the aircraft for surveillance purposes.
10. Flight Number Landing (INT): Flight number for the landing.
11. Regional or International Flight: Indicates whether the flight is regional or international.
12. Date: Date of the flight.
13. Entry Point: Point of entry into a specific airspace or region.


I had to combine all the datafiles in one frame so i took this [strategy](https://www.kaggle.com/code/crawford/loading-all-769-csvs-into-one-dataframe/notebook) for this step

In [2]:
import numpy as np
import pandas as pd
import glob
import os

In [3]:
# See how many files there are in the directory. 
# "!" commands are called "magic commands" and allow you to use bash
file_dir = '../input/asecnasj2223'
!ls $file_dir


20220927.csv  20221124.csv  20230110.csv  20230218.csv	20230329.csv
20220928.csv  20221125.csv  20230111.csv  20230219.csv	20230330.csv
20220929.csv  20221126.csv  20230112.csv  20230220.csv	20230331.csv
20220930.csv  20221127.csv  20230113.csv  20230221.csv	20230401.csv
20221001.csv  20221128.csv  20230114.csv  20230222.csv	20230402.csv
20221002.csv  20221129.csv  20230115.csv  20230223.csv	20230403.csv
20221003.csv  20221130.csv  20230116.csv  20230224.csv	20230404.csv
20221004.csv  20221201.csv  20230117.csv  20230225.csv	20230405.csv
20221005.csv  20221202.csv  20230118.csv  20230226.csv	20230406.csv
20221006.csv  20221203.csv  20230119.csv  20230227.csv	20230407.csv
20221007.csv  20221204.csv  20230120.csv  20230228.csv	20230408.csv
20221008.csv  20221205.csv  20230121.csv  20230301.csv	20230409.csv
20221010.csv  20221206.csv  20230122.csv  20230302.csv	20230410.csv
20221011.csv  20221207.csv  20230123.csv  20230303.csv	20230411.csv
20221012.csv  20221208.csv  20230124.csv  202303

In [4]:
# Number of files we are dealing with
!ls $file_dir | wc -l

194


In [5]:
files = glob.glob(os.path.join(file_dir, "*.csv"))

All my file where displayed as one column and not 18, I use this code to make sure it work and then loop over the 196 file

In [6]:
import pandas as pd

data_file = '/kaggle/input/asecnasj2223/20220927.csv'

data = pd.read_csv(data_file, header=None)

df = pd.DataFrame({'Data': data[0]})
df[['Line Number',             # Identification number for the flight record.
              'Immatriculation',         # Aircraft registration number.
              'Avion Type',               # Type of aircraft used for the flight.
              'Turbulence Type ',  # Categorization of turbulence intensity.
              'Departure Emplacement (ICAO code)',   # ICAO code representing the departure location.
              'Destination Emplacement (ICAO code)',  # ICAO code representing the destination location.
              'Flight Rule',              # Flight rules followed: VFR-VMC or IFR-IMC.
              'Radioborn Maker - Avion Capacity (COMNAV)',    # Manufacturer of communication and navigation equipment.
              'Radioborn Maker - Avion Capacity (Surveillance)',   # Manufacturer of surveillance equipment.
              'Flight Number Landing (INT)',  # Flight number for the landing.
              'Regional or International Flight',   # Indicates if the flight is regional or international.
              'Date',                     # Date of the flight.
              'Entry Point',              # Point of entry into a specific airspace or region.
              'Hour of Entry',            # Time of entry into the specified airspace or region.
              'Flight Level',             # Altitude at which the aircraft is flying.
              'Exit Point',               # Point of exit from a specific airspace or region.
              'Hour of Exit',             # Time of exit from the specified airspace or region.
              'Progression of the Flight'  # Description of the flight's progression or trajectory.
   ]] = df['Data'].str.split('\s+', expand=True, n=17)

df = df.drop('Data', axis=1)

df.shape


(107, 18)

In [7]:


# List of file paths matching a specific pattern
file_paths = glob.glob('/kaggle/input/asecnasj2223/*.csv')

# Empty list to store the DataFrames
dfs = []

# Iterate over each file path
for file_path in file_paths:
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file_path, header=None)
    df = pd.DataFrame({'Data': data[0]})
    df[['Line Number', 'Immatriculation', 'Avion Type', 'Turbulence Type', 'Departure Emplacement (ICAO code)',
        'Destination Emplacement (ICAO code)', 'Flight Rule', 'Radioborn Maker - Avion Capacity (COMNAV)',
        'Radioborn Maker - Avion Capacity (Surveillance)', 'Flight Number Landing (INT)',
        'Regional or International Flight', 'Date', 'Entry Point', 'Hour of Entry', 'Flight Level',
        'Exit Point', 'Hour of Exit', 'Progression of the Flight']] = df['Data'].str.split('\s+', expand=True, n=17)
    df = df.drop('Data', axis=1)
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate the list of DataFrames into a single DataFrame
combined_df = pd.concat(dfs)

# Reset the index of the combined DataFrame
combined_df = combined_df.reset_index(drop=True)

combined_df.head()


,Line Number,Immatriculation,Avion Type,Turbulence Type,Departure Emplacement (ICAO code),Destination Emplacement (ICAO code),Flight Rule,Radioborn Maker - Avion Capacity (COMNAV),Radioborn Maker - Avion Capacity (Surveillance),Flight Number Landing (INT),Regional or International Flight,Date,Entry Point,Hour of Entry,Flight Level,Exit Point,Hour of Exit,Progression of the Flight
0,MAU855,3BNBU,A339,H,FIMP,FAOR,I,SDE3FGHIJ3J4J5M1RWY,LB1D1,OVF,INTERNATIONAL,19700101,GERAG,2117,400,SUNIR,2254,400 /NANTA;2127;400/DUPTO;2154;400/TU;2226;40...
1,KQA263,5YFFB,E190,M,FMMI,HKJK,I,SCDE2E3FGHIRWYZ,LB1,DEP,INTERNATIONAL,20221209,FMMI,2340,360,KINAN,0107,360 /TNV;2341;58/ZAPR;2343;78/1838S04726E;234...
2,FBU702,FHMIX,A359,H,LFPO,FMEE,I,ABDE1E2E3GHIJ1J2J3J4J5LM1OP2RVWXYZ,LB1D1,OVF,INTERNATIONAL,19700101,DENLI,2356,390,UVENA,0102,390 /ZCRNA;2356;350/IXEBU;0012;390/1225S05132...
3,CFG2314,DAIYD,A332,H,EDDF,FIMP,I,SDE1E2E3FGHIJ5LM1OP2RVWXYZ,LB1D1,OVF,INTERNATIONAL,19700101,ATOLA,0036,390,APLEM,0202,390 /ZCRNA;0036;390/BIRAL;0046;390/DO;0103;39...
4,AFR652,FGSQX,B77W,H,LFPG,FMEE,I,SDE2E3FGHIJ3J5M1P2RWXYZ,LB1D1,OVF,INTERNATIONAL,19700101,DENLI,0104,350,UVENA,0212,350 /ZCRNA;0104;370/IXEBU;0121;370/MIDRI;0131...


In [8]:
# Export the combined DataFrame to a CSV file
combined_df.to_csv('combined_data.csv', index=False)